## Machine Learning
El modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno.

### Pre-procesamiento

 Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo

In [60]:
# Importamos librerias 
import pandas as pd 
import numpy as np
import seaborn as sns 
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Traemos los datos 
reviews =pd.read_parquet("D:\\Documentos\\Courses\\Henry\\Henry Students\\Labs\\Cohorte 07\\PI_1\\PI MLOps - STEAM\\Data\\parquet\\Datos Generales\\reviews.parquet")
items = pd.read_parquet("D:\Documentos\Courses\Henry\Henry Students\Labs\Cohorte 07\PI_1\PI MLOps - STEAM\Data\parquet\\Datos Generales\items.parquet")


In [3]:
# Tener un punto de reinicio
df_reviews = pd.DataFrame(reviews)
df_items = pd.DataFrame(items)

In [13]:
# Inicializamos el analizador de sentimientos de NLTK 
sen = SentimentIntensityAnalyzer()

In [19]:
# Creamos la funcion
# Donde 0 = Negativo; 1 = Neutral; 2 = Positivo
def sentiment_analysis(text):
    if pd.isnull(text):
        return 1 
    elif isinstance(text, str):
        sentiment = sen.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2 #Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1 # Neutral
    else:
        return 1 # Neutral -> no son tipo str

In [25]:
# Cambiamos la columna a tipo str
df_reviews['review'] = df_reviews['review'].astype(str)
#Aplicamos la funcion 
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(sentiment_analysis)
df_reviews

,user_id,user_url,posted,item_id,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D,2


In [26]:
#Eliminamos la columna review
df_reviews = df_reviews.drop(columns=['review'])

In [27]:
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend',
       'sentiment_analysis'],
      dtype='object')

##### Parquet

In [28]:
df_reviews.to_parquet("reviews.parquet")

##### Se puede aplicar algunas tecnicas mas pero por cuestion tiempo aqui nos quedaremos mas adelante agregare mas <div> Seguiremos con el ML 

##### Necesitaremos, “ítems” y las valoraciones de los usuarios. Los ítems pueden ser, canciones, películas, productos, ó lo que sea que queremos recomendar. <div> En nuestro caso son videojuegos en Steam

In [59]:
# Importamos librerias de ML 
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import matplotlib.pylab as plt

In [24]:
# Traemos los dataset de datos 
df_reviews = pd.DataFrame(reviews)
df_items = pd.DataFrame(items)

In [6]:
# Veamos las columna que hay 
print(df_reviews.columns)
print(df_items.columns)

Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend',
       'sentiment_analysis'],
      dtype='object')
Index(['user_id', 'user_url', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')


In [47]:
# Veamos cuantos usarios tenemos y cuantos juegos tenemos
n_items_id = df_items.item_id.unique().shape[0]
n_items = df_items.user_id.unique().shape[0]
print(str(n_items) + ' user')
print(str(n_items_id) + ' Juegos')

68403 user
10050 Juegos


In [25]:
# Eliminamos las columnas no deseadas y luego Hacemos el metodo merge para unificar las columnas 
df_items  = df_items.drop(columns= ['user_id', 'user_url', 'item_name', 'playtime_2weeks'])
df_reviews= df_reviews.drop(columns=['user_id', 'user_url', 'recommend', 'posted'])
print(f"columnas de df_items {df_items.columns}, columnas de df_reviews{df_reviews.columns}") 

columnas de df_items Index(['item_id', 'playtime_forever'], dtype='object'), columnas de df_reviewsIndex(['item_id', 'sentiment_analysis'], dtype='object')


In [26]:
print(df_items.shape[0])
print(df_reviews.shape[0])

3285249
59305


In [27]:
total_rows_df_reviews = len(df_reviews)
total_rows_df_items = len(df_items)
print(total_rows_df_items, total_rows_df_reviews)

3285249 59305


In [28]:
# Vamos hacer un muestro aleatorio de filas porque mi maquina no pudo con tantos datos 
# Definir el tamaño de muestra proporcional al tamaño de cada DataFrame
sample_size_items = 30000  # El mismo tamaño que df_reviews
sample_size_reviews = 30000

# Seleccionar una muestra aleatoria de tamaño proporcional para df_items
sample_df_items = df_items.sample(n=sample_size_items, replace=False, random_state=42)

# Seleccionar una muestra aleatoria de tamaño proporcional para df_reviews
sample_df_reviews = df_reviews.sample(n=sample_size_reviews, replace=False, random_state=42)


In [29]:
# Seleccionamos las columnas que queremos para crear la matriz 
items_reviews = pd.merge(sample_df_items, sample_df_reviews[['item_id', 'sentiment_analysis']], on='item_id', how='inner')

In [47]:
data_columna1 = ['item_id', 'sentiment_analysis', 'playtime_forever']
filtrado = items_reviews[data_columna1]
filtrado

,item_id,sentiment_analysis,playtime_forever
0,102600,0,10
1,102600,2,10
2,102600,2,10
3,102600,2,10
4,102600,0,17
...,...,...,...
2803783,410210,0,344
2803784,410210,2,344
2803785,485380,0,31
2803786,485380,0,31


In [48]:
# Cambiamos los tipos de datos de las columnas a str
filtrado[data_columna1] = filtrado[data_columna1].astype(str)
filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2803788 entries, 0 to 2803787
Data columns (total 3 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   item_id             object
 1   sentiment_analysis  object
 2   playtime_forever    object
dtypes: object(3)
memory usage: 64.2+ MB


In [49]:
# Definimos una función para mezclar columnas en una
def mezclar_en_columna(df):
    return df["playtime_forever"]+ "" +df["item_id"]+ " "+ df["sentiment_analysis"]

In [50]:
# Creamos una columna llamada "datos_agrupado" que integra todas las demas
filtrado["datos_agrupado"]=filtrado.apply(mezclar_en_columna, axis=1)
filtrado

,item_id,sentiment_analysis,playtime_forever,datos_agrupado
0,102600,0,10,10102600 0
1,102600,2,10,10102600 2
2,102600,2,10,10102600 2
3,102600,2,10,10102600 2
4,102600,0,17,17102600 0
...,...,...,...,...
2803783,410210,0,344,344410210 0
2803784,410210,2,344,344410210 2
2803785,485380,0,31,31485380 0
2803786,485380,0,31,31485380 0


In [52]:
modelo  = modelo = TfidfVectorizer(stop_words='english')
# Crear una matriz de utilidad utilizando fit_trasnform
df_matrix = modelo.fit_transform(filtrado[data_columna1])

In [53]:
modelo

TfidfVectorizer(stop_words='english')

In [54]:
df_matrix

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [61]:
# Calculamos la similitud del coseno tomando todos las filas de la matriz
similitud_del_coseno = cosine_similarity(X = df_matrix, Y = df_matrix)
similitud_del_coseno

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [63]:
df_similitud_del_coseno = pd.DataFrame(similitud_del_coseno)

In [64]:
# Guardamos el dataframe similitud de coseno 
df_similitud_del_coseno.to_parquet("similitud del coseno.parquet")

In [65]:
# Reiniciamos el indice del dataframe y creamos una serie de índices
filtrado = filtrado.reset_index()
indice = pd.Series(filtrado.index, index=filtrado["item_id"])

In [66]:
# Revisamos las dimensiones
filtrado.shape

(2803788, 5)

In [71]:
# Vamos hacer un muestro aleatorio de filas porque render tal vez pueda con tantos datos 
# Definir el tamaño de muestra proporcional al tamaño de cada DataFrame
sample_size_filtrado = 100000  


# Seleccionar una muestra aleatoria de tamaño proporcional para df_items
sample_df_filtrado = filtrado.sample(n=sample_size_filtrado, replace=False, random_state=42)

In [81]:
# Revisamos las dimensiones
sample_df_filtrado = sample_df_filtrado.reset_index()

In [83]:
# ELiminamos las columnas index y level 0
sample_df_filtrado = sample_df_filtrado.drop(columns=['index'])


In [84]:
sample_df_filtrado

,item_id,sentiment_analysis,playtime_forever,datos_agrupado
0,33930,2,19653,1965333930 2
1,236390,2,2123,2123236390 2
2,230410,2,112,112230410 2
3,4000,2,121,1214000 2
4,224260,2,50,50224260 2
...,...,...,...,...
99995,730,2,95235,95235730 2
99996,4000,2,173,1734000 2
99997,252950,2,90,90252950 2
99998,550,0,1,1550 0


In [85]:
# Guardamos los datos 
sample_df_filtrado.to_parquet("data_modelo.parquet")
indice.to_csv('indices.csv')